In [29]:
import pickle

data_list = [211,240,269,329,366,427,493,496,833,834,835,1009,1092,1175,1506,1586,1872,2508,2625]
for data_number in data_list:
    with open('../data/both57m_rigged/%d.pkl' % data_number, 'rb') as f:
        data = pickle.load(f)
    
    full_annot = data['full_annot']
    print(data_number, full_annot,"\n")

211 ['a person clenches their fist and extends their thumb downwards to indicate disapproval.'
 'a person extends their thumb downwards to indicate dissatisfaction.'
 'a person expresses displeasure by pushing both arms down.'] 

240 ['a person extends their finger and points while shouting loudly to express uproar.'
 'a person extends their hand and shakes it to indicate provocation.'
 'a person expresses arrogance by loudly shouting and pointing at others.'] 

269 ['a person swiftly closes their palms in the air to indicate enjoyment.'
 'a person repeatedly makes a clapping motion with one hand in the air, indicating they are enjoying it and not getting tired of it.'
 'when a person expresses being happy with something, they will repeatedly clap their hands in the air.'] 

329 ['a person touches their chin to indicate that they are thinking.'
 "a person extends their finger and points back and forth towards themselves to mean 'pay attention to me'."
 'a person expresses their closene

In [30]:
import torch,sys,pickle
import numpy as np
sys.path.append('../utils')
from mujoco_parser import MuJoCoParserClass
from util import rpy2r,get_idxs,np_uv

comp_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
xml_path = '../asset/smplh_rig/scene_smplh_rig.xml'
env = MuJoCoParserClass(name='Common-Rig',rel_xml_path=xml_path,VERBOSE=False)
ls_y = env.get_p_body('left_shoulder')[1]
rs_y = env.get_p_body('right_shoulder')[1]

# Modify the color of body exluding 'world'
for body_name in env.body_names:
    if body_name in ['world']: continue
    for geom_idx in env.get_geom_idxs_from_body_name(body_name=body_name):
        env.model.geom(geom_idx).rgba = [0.3,0.3,0.5,0.5]
        
video = []
tick = 0

ZERO_ROOT = True

# data_number = 834
# with open('../data/both57m_rigged/%d.pkl' % data_number, 'rb') as f:
#     data = pickle.load(f)

# qpos_data = data['qpos']
# seq_len = data['length']

# data_number = 835
# with open('../data/both57m_rigged/%d.pkl' % data_number, 'rb') as f:
#     data = pickle.load(f)

# qpos_rig_data = np.concatenate((qpos_data, data['qpos']), axis=0)
# seq_len += data['length']

data_number = 240
with open('../data/both57m_rigged/%d.pkl' % data_number, 'rb') as f:
    data = pickle.load(f)

qpos_rig_data = data['qpos']
seq_len = data['length']

start_tick = 25
end_tick = seq_len
final_len = end_tick - start_tick

ls2le_array = np.zeros((final_len,3))
le2lw_array = np.zeros((final_len,3))
rs2re_array = np.zeros((final_len,3))
re2rw_array = np.zeros((final_len,3))
lw_rot_array = np.zeros((final_len,3,3))
rw_rot_array = np.zeros((final_len,3,3))
lh_array = np.zeros((final_len,6))
rh_array = np.zeros((final_len,6))

while tick < end_tick-start_tick:
    q_rig = qpos_rig_data[tick+start_tick,:]

    if ZERO_ROOT:
        q_rig[3:7] = 0
    
    env.forward(q=q_rig,INCREASE_TICK=True)
    ls = env.get_p_body('left_shoulder')
    le = env.get_p_body('left_elbow')
    lw = env.get_p_body('left_wrist')
    rs = env.get_p_body('right_shoulder')
    re = env.get_p_body('right_elbow')
    rw = env.get_p_body('right_wrist')

    ls2le_array[tick,:] = np_uv(le-ls)
    le2lw_array[tick,:] = np_uv(lw-le)
    rs2re_array[tick,:] = np_uv(re-rs)
    re2rw_array[tick,:] = np_uv(rw-re)
    lw_rot_array[tick,:] = env.get_R_body('left_wrist')
    rw_rot_array[tick,:] = env.get_R_body('right_wrist')
    lh_array[tick,:] = q_rig[[46,47,31,34,42,38]]
    rh_array[tick,:] = q_rig[[73,74,58,61,69,65]]

    tick = tick + 1

In [31]:
SAVE_VIDEO = True
vis_tick = 0

# Initialize MuJoCo viewer
env.init_viewer(viewer_title='SMPLH-Rig',viewer_width=800,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=180,distance=3.0,elevation=-20,lookat=[0.02,-0.03,1.2])
env.reset()

while vis_tick < final_len:
    q_rig = qpos_rig_data[vis_tick+start_tick,:]
    env.forward(q=q_rig,INCREASE_TICK=True)

    # Plot world frame
    env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
    env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3,3),
                PLOT_AXIS=False,label="tick:[%d/%d]"%(vis_tick,final_len))
    env.render()  
    frame = env.grab_image()
    video.append(frame)

    for rev_joint_idx,rev_joint_name in zip(env.rev_joint_idxs,env.rev_joint_names):
        axis_joint = env.model.jnt_axis[rev_joint_idx]
        p_joint,R_joint = env.get_pR_joint(joint_name=rev_joint_name)
        axis_world = R_joint@axis_joint
        axis_rgba = np.append(np.eye(3)[:,np.argmax(axis_joint)],0.2)
        axis_len,axis_r = 0.02,0.002
        env.plot_arrow_fr2to(
            p_fr=p_joint,p_to=p_joint+axis_len*axis_world,
            r=axis_r,rgba=axis_rgba)
        
    # for body_name in ['left_wrist','right_wrist']:
    #     p,R = env.get_pR_body(body_name=body_name)
    #     env.plot_T(p=p,R=R,PLOT_AXIS=True,axis_len=0.1,axis_width=0.005)

    vis_tick = vis_tick + 1
    if vis_tick == final_len:
        if not SAVE_VIDEO: tick = 0

env.close_viewer()

if SAVE_VIDEO:
    import mediapy as media
    media.show_video(images=video,fps=15,width=700)

This browser does not support the video tag.


In [32]:
recon_tick = 0

# Initialize MuJoCo viewer
env.init_viewer(viewer_title='SMPLH-Rig',viewer_width=800,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=180,distance=3.0,elevation=-20,lookat=[0.02,-0.03,1.2])
env.reset()

BODY_NAMES = ['left_elbow','left_wrist','right_elbow','right_wrist']
JOINT_NAMES = ['l_shoulder1', 'l_shoulder2', 'l_shoulder3', 'l_elbow', 'l_wrist1', 'l_wrist2', 'l_wrist3',
               'r_shoulder1', 'r_shoulder2', 'r_shoulder3', 'r_elbow', 'r_wrist1', 'r_wrist2', 'r_wrist3']

coupled_joint_idxs_list = [[47,48],[31,32,33],[34,35,36],[42,43,44],[38,39,40],[41,37],
                           [74,75],[58,59,60],[61,62,63],[65,66,67],[69,70,71],[68,64]]
coupled_joint_weights_list = [[1,1],[1,3,2],[1,3,2],[1,3,2],[1,3,2],[2,5],
                              [1,1],[1,3,2],[1,3,2],[1,3,2],[1,3,2],[2,5]]
hand_indices = [27,28,29,54,55,56,45,30,41,46,47,31,
                34,42,38,72,57,68,73,74,58,61,69,65]
video = []

while recon_tick < final_len:
    p_target = {}
    p_target['left_elbow'] = ls + len_ls2le * ls2le_array[recon_tick,:]
    p_target['left_wrist'] = p_target['left_elbow'] + len_le2lw * le2lw_array[recon_tick,:]
    p_target['right_elbow'] = rs + len_rs2re * rs2re_array[recon_tick,]
    p_target['right_wrist'] = p_target['right_elbow'] + len_re2rw * re2rw_array[recon_tick,:]
    
    idxs = get_idxs(env.rev_joint_names,JOINT_NAMES)
    q_mins_body,q_maxs_body = env.rev_joint_mins[idxs],env.rev_joint_maxs[idxs]
    joint_idxs_fwd_body = env.get_idxs_fwd(joint_names=JOINT_NAMES)
    joint_idxs_jac_body = env.get_idxs_jac(joint_names=JOINT_NAMES)

    q_body = env.get_qpos_joints(JOINT_NAMES)
    body_diff = 1e3
    prev_q_pos = env.data.qpos.copy()

    while body_diff > 1e-4:
        for ik_tick in range(1):
            J_list,ik_err_list = [],[]
            for ik_body_name in BODY_NAMES:
                ik_p_trgt = p_target[ik_body_name]
                ik_R_trgt = None
                IK_P = True
                IK_R = False
                J,ik_err = env.get_ik_ingredients(
                    body_name=ik_body_name,p_trgt=ik_p_trgt,R_trgt=ik_R_trgt,
                    IK_P=IK_P,IK_R=IK_R)
                J_list.append(J)
                ik_err_list.append(ik_err)
            J_stack      = np.vstack(J_list)
            ik_err_stack = np.hstack(ik_err_list)
            dq = env.damped_ls(J_stack,ik_err_stack,stepsize=1,eps=1e-2,th=np.radians(1.0))
            q_body = q_body + dq[joint_idxs_jac_body]

            q_body = np.clip(q_body,q_mins_body,q_maxs_body)
            env.forward(q=q_body,joint_idxs=joint_idxs_fwd_body)

            q_env = env.data.qpos.copy()
            body_diff = np.linalg.norm(q_env - prev_q_pos)
            prev_q_pos=q_env.copy()

    q_rig = qpos_rig_data[recon_tick,:]
    q_env[hand_indices] = q_rig[hand_indices]
    for i in range(len(coupled_joint_idxs_list)):
        coupled_joint_idx = coupled_joint_idxs_list[i]
        coupled_joint_weights = coupled_joint_weights_list[i]

        for j in range(len(coupled_joint_idx)):
            q_env[coupled_joint_idx[j]] = q_env[coupled_joint_idx[0]] * coupled_joint_weights[j] / coupled_joint_weights[0]
     
    env.forward(q=q_env,INCREASE_TICK=True)

    env.plot_T(p=np.zeros(3),R=np.eye(3),label="Tick: [%d/%d]"%(recon_tick,final_len),axis_len=0.5,axis_width=0.005)
    env.render()

    video.append(env.grab_image())
    recon_tick += 1

env.close_viewer()
media.show_video(images=video,fps=15,width=700)

This browser does not support the video tag.


In [33]:
xml_path = '../asset/kimlab_dualarms/scene_dualarms.xml'
env = MuJoCoParserClass(name='Dual Arms',rel_xml_path=xml_path,VERBOSE=False)
env.set_geom_color(
    rgba                = (0.2,0.2,0.2,0.9),
    body_names_to_color = [
        'robot1/end_link','robot1/index_L1','robot1/index_L2','robot1/index_tip',
        'robot1/middle_L1','robot1/middle_L2','robot1/middle_tip',
        'robot1/ring_L1','robot1/ring_L2','robot1/ring_tip',
        'robot1/pinky_L1','robot1/pinky_L2','robot1/pinky_tip',
        'robot1/thumb_L1','robot1/thumb_L2','robot1/thumb_tip',
        'robot2/end_link','robot2/index_L1','robot2/index_L2','robot2/index_tip',
        'robot2/middle_L1','robot2/middle_L2','robot2/middle_tip',
        'robot2/ring_L1','robot2/ring_L2','robot2/ring_tip',
        'robot2/pinky_L1','robot2/pinky_L2','robot2/pinky_tip',
        'robot2/thumb_L1','robot2/thumb_L2','robot2/thumb_tip'],
)
print ("Done.")

Done.


In [34]:
env.init_viewer(viewer_title='Dual Arms',viewer_width=800,viewer_height=800,viewer_hide_menus=True)
env.update_viewer(azimuth=180,distance=3.0,elevation=-20,lookat=[0.02,-0.03,1.2])
env.reset()

BODY_NAMES = ['robot1/end_link','robot2/end_link']
JOINT_NAMES = ['robot1/joint1','robot1/joint2','robot1/joint3','robot1/joint4','robot1/joint5','robot1/joint6',
               'robot2/joint1','robot2/joint2','robot2/joint3','robot2/joint4','robot2/joint5','robot2/joint6']

ls = env.get_p_body('robot1/link2')
le = env.get_p_body('robot1/link4')
lw = env.get_p_body('robot1/link6')
rs = env.get_p_body('robot2/link2')
re = env.get_p_body('robot2/link4')
rw = env.get_p_body('robot2/link6')

len_ls2le = np.linalg.norm(le-ls)
len_le2lw = np.linalg.norm(lw-le)
len_rs2re = np.linalg.norm(re-rs)
len_re2rw = np.linalg.norm(rw-re)

q_env_array = np.zeros((final_len,env.model.nq))

for tick in range(final_len):
    p_target = {}
    p_target['robot1/end_link'] = ls + len_ls2le * ls2le_array[tick,:] + len_le2lw * le2lw_array[tick,:]
    p_target['robot2/end_link'] = rs + len_rs2re * rs2re_array[tick,:] + len_re2rw * re2rw_array[tick,:]
    R_target = {}
    R_target['robot1/end_link'] = lw_rot_array[tick,:] @ rpy2r(np.radians([0,0,90]))
    R_target['robot2/end_link'] = rw_rot_array[tick,:] @ rpy2r(np.radians([0,0,-90]))

    idxs = get_idxs(env.rev_joint_names,JOINT_NAMES)
    q_mins_body,q_maxs_body = env.rev_joint_mins[idxs],env.rev_joint_maxs[idxs]
    joint_idxs_fwd_body = env.get_idxs_fwd(joint_names=JOINT_NAMES)
    joint_idxs_jac_body = env.get_idxs_jac(joint_names=JOINT_NAMES)

    q_body = env.get_qpos_joints(JOINT_NAMES)
    body_diff = 1e3
    prev_q_pos = env.data.qpos.copy()

    while body_diff > 1e-4:
        for ik_tick in range(10):
            J_list,ik_err_list = [],[]
            for ik_body_name in BODY_NAMES:
                ik_p_trgt = p_target[ik_body_name]
                ik_R_trgt = R_target[ik_body_name]
                IK_P = True
                IK_R = True
                J,ik_err = env.get_ik_ingredients(
                    body_name=ik_body_name,p_trgt=ik_p_trgt,R_trgt=ik_R_trgt,
                    IK_P=IK_P,IK_R=IK_R)
                J_list.append(J)
                ik_err_list.append(ik_err)
            J_stack      = np.vstack(J_list)
            ik_err_stack = np.hstack(ik_err_list)
            dq = env.damped_ls(J_stack,ik_err_stack,stepsize=1,eps=1e-2,th=np.radians(1.0))
            q_body = q_body + dq[joint_idxs_jac_body]

            q_body = np.clip(q_body,q_mins_body,q_maxs_body)
            env.forward(q=q_body,joint_idxs=joint_idxs_fwd_body)

            q_env = env.data.qpos.copy()
            body_diff = np.linalg.norm(q_env - prev_q_pos)
            prev_q_pos=q_env.copy()

    q_env_array[tick,:] = q_env

env.close_viewer()

In [35]:
import mediapy

env.init_viewer(viewer_title='Dual-Arm',viewer_width=800,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=180,distance=3.0,elevation=-20,lookat=[0.02,-0.03,1.2])
env.reset()

SAVE_VIDEO = True

video = []
plot_tick = 0

coupled_joint_idxs_list = [
    [6,7],[8,9],[10,11],[12,13],
    [22,23],[24,25],[26,27],[28,29]
]
coupled_joint_weights_list = [
    [1,3],[1,3],[1,3],[1,3],
    [1,3],[1,3],[1,3],[1,3]
]

while plot_tick < final_len and env.is_viewer_alive():
    p_target = {}
    p_target['robot1/end_link'] = ls + len_ls2le * ls2le_array[plot_tick,:] + len_le2lw * le2lw_array[plot_tick,:]
    p_target['robot2/end_link'] = rs + len_rs2re * rs2re_array[plot_tick,:] + len_re2rw * re2rw_array[plot_tick,:]
    R_target = {}
    R_target['robot1/end_link'] = lw_rot_array[plot_tick,:] 
    R_target['robot2/end_link'] = rw_rot_array[plot_tick,:] @ rpy2r(np.radians([0,0,-90]))

    q_robot = q_env_array[plot_tick,:]
    q_robot[[14,15,6,8,10,12]] = lh_array[plot_tick,:] * -1
    q_robot[[30,31,22,24,26,28]] = rh_array[plot_tick,:] * [-1,-1,1,1,1,1]
    
    for i in range(len(coupled_joint_idxs_list)):
        coupled_joint_idxs = coupled_joint_idxs_list[i]
        coupled_joint_weights = coupled_joint_weights_list[i]
        for j in range(1,len(coupled_joint_idxs)):
            q_robot[coupled_joint_idxs[j]] = q_robot[coupled_joint_idxs[0]] * coupled_joint_weights[j] / coupled_joint_weights[0]
    env.forward(q=q_robot,INCREASE_TICK=True)

    env.plot_T(p=np.zeros(3),R=np.eye(3),label="Tick: [%d/%d]"%(plot_tick,final_len),axis_len=0.5,axis_width=0.005)
    env.render()
    
    # for body_name in ['robot1/end_link','robot2/end_link']:
    #     env.plot_T(p=p_target[body_name],R=R_target[body_name],PLOT_AXIS=True,axis_len=0.15,axis_width=0.005)
    #     env.plot_T(p=env.get_p_body(body_name),R=env.get_R_body(body_name),PLOT_AXIS=True,axis_len=0.2,axis_width=0.01)

    video.append(env.grab_image())
    plot_tick += 1
    if plot_tick == final_len:
        if not SAVE_VIDEO: plot_tick = 0

env.close_viewer()
if SAVE_VIDEO:mediapy.show_video(images=video,fps=15,width=700)

This browser does not support the video tag.
